# Load Library & Preprocess Data 

In [ ]:
import os
GPU = f'0'
os.environ['CUDA_VISIBLE_DEVICES']=GPU
import random
from glob import glob
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)
from tensorflow.keras.applications import (
    MobileNet,
    MobileNetV2,
    EfficientNetB7
)
from tensorflow.keras.layers import (
    GlobalAveragePooling2D,
    Dense
)
from sklearn.model_selection import train_test_split

#### Set Seed 

In [ ]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

#### Config 

In [ ]:
batch_size = 64

classes = 1049
size = 600

learning_rate = 1e-2
wd = 0.0005
max_lr = 1e-2
min_lr = 5e-5
cycle_len = 20

EPOCHS = 2

#### Load Data 

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
df['landmark_id'] = df['landmark_id'].astype(str)

#### Split Train & Valid 

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=SEED)

In [ ]:
train_df.shape, val_df.shape

# Datagenerator 

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='id',
    y_col='landmark_id',
    target_size=(size,size),
    batch_size=batch_size,
)
val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='id',
    y_col='landmark_id',
    target_size=(size,size),
    batch_size=batch_size,
)

# Build Model 

In [ ]:
with tf.device(f'/device:GPU:{GPU}'):
    base_model = EfficientNetB7(
        input_shape=(size,size,3),
        include_top=False,
        weights='imagenet',
    )
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dense(classes, activation='softmax')
    ])

model.summary()

# Compile 

In [ ]:
optimizer = tfa.optimizers.AdamW(learning_rate, wd)

model.compile(
    optimizer = optimizer,
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

#### callbacks 

In [ ]:
checkpoint_filepath = './checkpoints/mobilenetv2/'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False,
                                                monitor='val_accuracy', mode='max', save_best_only=True,
                                        verbose=1)

def trianfle_fn(x):
    return 1. / (2.**(x - 1))
clr_f = tfa.optimizers.CyclicalLearningRate(
    initial_learning_rate = max_lr,
    maximal_learning_rate = min_lr,
    step_size = cycle_len,
    scale_fn = trianfle_fn
)

learing_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(clr_f)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, 
                                          verbose=2, mode='auto', baseline=None, 
                                          restore_best_weights=True)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

# Callbacks
callbacks = [checkpoint, learing_rate_scheduler, early_stop, tensorboard_callback]

# delete checkpoint
# callbacks = [learing_rate_scheduler, early_stop, tensorboard_callback]

# delete EarlyStop
# callbacks = [checkpoint, learing_rate_scheduler, tensorboard_callback]

# Only Scheduler
# callbacks = [learing_rate_scheduler, tensorboard_callback]

# Training 

In [ ]:
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
)